# Астероиды и кометы

## Заложим основы

In [97]:
import numba as nb
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import unittest
from pathlib import Path
from PIL import Image
from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
from IPython import display
from tqdm import tqdm
from dataclasses import dataclass

dt = 1e-3
G = 6.7e-11
au = 149597870700

InteractiveShell.ast_node_interactivity = "all"

### Зададим объект
В моей программе только один класс, поскольку принципиальных различий между реальными небесными телами нет

In [59]:
@dataclass
class CosmicBody:   
    mass: float
    obj_radius: float
    radius_vec: np.array
    velocity_vec: np.array


### Зададим методы
Их будет всего два:
- Рассчёт изменений всех параметров системы за dt (учитывает 2 возможных сценария: смещение и столкновение)
- Определение типа траектории

In [116]:
def dtChange(syst):
    for i in range(len(syst)):
        syst[i].radius_vec += np.dot(dt, syst[i].velocity_vec)

        
    for i in range(len(syst)):
        for j in range(len(syst)):
            if j != i:
                dr = np.array(syst[j].radius_vec) - np.array(syst[i].radius_vec)
                syst[i].velocity_vec += np.dot(G*dt/(np.linalg.norm(dr)**3)*syst[j].mass, dr)
    
    for i in range(len(syst)):
        for j in range(len(syst)) :
            if syst[i].mass >= syst[j].mass:
                dr = np.array(syst[j].radius_vec) - np.array(syst[i].radius_vec)
                if np.linalg.norm(dr) < syst[i].obj_radius:
                    syst[i].mass += syst[j].mass
                    syst[i].radius_vec += np.dot(syst[j].mass/(syst[j].mass+syst[i].mass), dr)
                    syst[i].obj_radius *= (1 + (syst[i].mass / syst[j].mass)) ** (1/3)
                    syst[j].mass = 0
                    syst[j].obj_radius = 0
                    syst[j].velocity_vec = 0
                    syst[j].radius_vec = 0

### Тесты

In [117]:
Big   = CosmicBody(100, 10, [0,0,0], [0, 0, 0])
Body1 = CosmicBody(0.5, 1,  [15, 1, 0],  [0, -0.5, 10])
Body2 = CosmicBody(20,  30, [0, 3, 50],  [20, -5, 0])
Body3 = CosmicBody(5,   2,  [10, 0, 12], [10, 0, -10])

for n in range(400):
    dtChange(syst=[Big, Body1, Body2, Body3])



C:\Users\ivang\AppData\Local\Temp/ipykernel_40636/2736731396.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  syst[i].velocity_vec += np.dot(G*dt/(np.linalg.norm(dr)**3)*syst[j].mass, dr)
C:\Users\ivang\AppData\Local\Temp/ipykernel_40636/2736731396.py:10: RuntimeWarning: invalid value encountered in double_scalars
  syst[i].velocity_vec += np.dot(G*dt/(np.linalg.norm(dr)**3)*syst[j].mass, dr)


### Визуализация